## Chapter 23: Modules and Tests

This chapter covers both modules and tests.  Modules are another great way to separate code and put types and common functions together in that they will be used.

#### 23.1: The Revise package

The `Revise` package is a great package to run to build packages.  Recall that since `struct`s are immutable, if you are trying to design a type, then you can't change it once you've made it.  This makes building a type and iterating on it difficult.  The `Revise` package handles this for you.  

In [ ]:
using Revise

#### 23.2: Creating a module

A `module` is a block of code that often contains:

* custom types (structs)
* functions
* other data needed by the functions and types. 

A great example is that of the Playing Cards that we've been using in a few chapters in this course.  To creating, here's a template:

```
module PlayingCards

end
```

We can then put all of the related playing card types and functions inside this module. 

Note: Julia convention is that a module name is in Pascal case.  

Instead of building this module in a jupyter cell, we'll often build it in a separate file.  This has already been done if you go to `../julia-files/PlayingCards.jl`, where `..` means up a directory.  You can open this inside jupyter. 

Then to load it, we will use the function `includet`, which is part of the `Revise` module.  The `t` stands for tracking in that the function `includet` will track changes to the module and reload as needed.

In [ ]:
includet("../julia-files/PlayingCards.jl")

And since it is a module, we need to give access to it with the `using` command.  Technically, the `PlayingCards` module is now in `Main` and can be loaded with either `using Main.PlayingCards` or the shortcut: 

In [ ]:
using .PlayingCards

Now we have access to all of the types and functions.

In [ ]:
h = Hand("2♡,6♣,2♠,2♢,6♢")

In [ ]:
isFullHouse(h)

If you notice, the module doesn't include the `runTrials` function we wrote in Chapter 15, so let's add that to the module.  (and don't forget to export it)

In [ ]:
"""
    runTrials(f::Function, trials::Integer)

For `trials` randomly selected hands, run the function `f` on each hand.  The fraction of hands where `f` is true is returned.

### Example
```julia-repl
runTrials(isFullHouse, 10_000_000)
```
"""
function runTrials(f::Function, trials::Integer)
  local deck=collect(1:52) # creates the array [1,2,3,...,52]
  local num_hands=0
  for i=1:trials
    shuffle!(deck)
    h = Hand(map(Card,deck[1:5])) # creates a hand of the first five cards of the shuffled deck
    if f(h)
      num_hands+=1
    end
  end
  num_hands/trials
end

In [ ]:
runTrials(isFullHouse,10_000_000)

#### Documenting functions in modules

Generally, use `?` before a type or function in Julia will show its documentation.  However, it doesn't appear to work within a notebook (still trying to track this down).  

In [ ]:
?isFullHouse

With the `Revise` module loaded and using `includet` on the file containing the module, you will have the ability to edit the julia file (module) and immediately changes are available without restarting the kernel.  This is very useful for creating a module.  

First, let's add a `isTwoPair` function to the module. Copy this from when we developed this in Chapter 20. 

In [ ]:
two_pair = Hand("T♠,T♠,Q♡,A♠,A♢")

In [ ]:
isTwoPair(two_pair)

### Exercise

Start a Geometry module by the following steps. 
1. create a file in VScode called `Geometry.jl`.
2. Put the template `module Geometry` then `end` in the file. 
3. Add your `struct` for `Point2D` from HW #5 inside the module.
4. Add the `Base.show` method for `Point2D`
4. Save the file
5. Load the file with `includet("Geometry.jl")`
6. Create a `Point2D` inside the notebook. 

#### 23.3: Unit Tests

A unit test is a way to ensure that code is written in a robust manner.  A test is a piece of code that will determine if something is returned true. In Julia, we will use the `Test` package

In [ ]:
using Test

A very simple example is if we define a variable

In [ ]:
n=3

and then test if this is 3 using the `@test` macro.

In [ ]:
@test n==3

We'll start by testing some of the `PlayingCards` types and function.  First, we can test if create a `Card` actually makes a card.

In [ ]:
@test isa(Card(1,4),Card)

which says yes, running the code `Card(1,4)` creates a Card object.  We can also test the other constructors as well:

In [ ]:
@test isa(Card(35),Card)

In [ ]:
@test isa(Card("3♣"),Card)

It's a good idea to also make sure that the checking routines work:

In [ ]:
@test isa(Card(13,5),Card)

However, notice that there's an error that the test wasn't passed as well as the error that is throw from the constructor.  A better way to do this is to test if an error is thrown:

In [ ]:
@test_throws ArgumentError Card(13,5)

##### Test Sets

A nice way to collect tests together in common ways is to make a test set.  The following covers all successfully constructed Cards:

In [ ]:
@testset "Legal Card Constructor" begin
        @test isa(Card(1,3),Card)
        @test isa(Card(45),Card)
        @test isa(Card("3\u2660"),Card)
        @test isa(Card("T♣"),Card)
end;

The idea of a test suite (many test sets that are associated with a module) is to have it on hand so when you update things, you don't break anything (have a regression). So we will put this in a file that we can just load

### Exercise
- Construct the following hands: full house, four of a kind, 2 pair and royal flush
- create a test suite on the  hands on `isFullHouse` and another test suite for `isRoyalFlush`.

In [ ]:
royal_flush = Hand("T♠,J♠,Q♠,K♠,A♠")
full_house = Hand("4♣,4♢,4♠,7♣,7♢")
four_of_a_kind = Hand("2♠,5♠,5♢,5♣,5♡")
two_pair = Hand("4♣,4♢,5♠,7♣,7♢")

The nice thing is to put all of your tests/test suites in a single file

In [ ]:
include("../julia-files/test-playing-cards.jl")

#### 23.5: A Rootfinding Module

We will also create a Rootfinding module that has some of the functions that we have seen over this course.  They are in the `Rootfinding.jl` file in the `julia-files` directory:

In [ ]:
includet("../julia-files/Rootfinding.jl")
using .Rootfinding

Take a look at that file:

In [ ]:
newton(x->x^2-2,1)

In [ ]:
newton(x->x^2+3,1, max_steps=19)

In [ ]:
x = newton(x->x^2-2,1)

It would be good to include some tests with this module.  Let's test if the root above is actually $\sqrt{2}$.

In [ ]:
@test x.root == sqrt(2)

And this fails because the two are not exactly equal.  Recall that floating point numbers are equal only all bits are equal.  Instead we will use the `isapprox` function which tests for equality within a tolerance (a bit complicated), see `?isapprox`.

In [ ]:
@test isapprox(x.root,sqrt(2))

In [ ]:
x = newton(x->x^2+1,2)

In [ ]:
@test !x.converged

In [ ]:
@testset "function with no root" begin
  val = newton(x->x^2+1,2)
  @test !val.converged
  @test val.num_steps == val.max_steps
 end;

### 23.6 Test Driven Development

The current thinking on developing code (either in Scientific Computing or other fields) is that of _Test Driven Development_ or TDD.  This basically flips the order of development.  One using TDD would first start with writing all of the test suites.  That is what you want everything to return.  After satisfied with this, you then write all of the functions and continually test while developing. 

### Example

If we would have used TDD to develop the Playing card code, we could write a function that goes through all 5-card hands (this isn't too bad to write) and for each hand type, determine the total number of hands (this is listed in the wikipedia article).  Then you can write all testing for types until getting the desired results. 